In [2]:
from PIL import Image
import numpy as np
import os

In [5]:
def resize(input_dir, output_dir, target_size=(32,32)):
    os.makedirs(output_dir, exist_ok=True)

    for kuva in os.listdir(input_dir):
        img_path = os.path.join(input_dir, kuva)
        img = Image.open(img_path)
        resized = img.resize(target_size, Image.BICUBIC)
        resized = resized.convert('RGB')
        resized.save(os.path.join(output_dir, kuva))

resize("./data/angry", "./data/resized_angry")
resize("./data/happy", "./data/resized_happy")
resize("./data/sad", "./data/resized_sad")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [16]:
import torch
from torchvision import transforms
import torch.nn.functional as F

angry_path = "./data/resized_angry"
happy_path = "./data/resized_happy"
sad_path = "./data/resized_sad"

valid_extensions = ('.jpg', '.jpeg', '.png')

data = []
labels = []

to_tensor = transforms.ToTensor()

for kuva in os.listdir(angry_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(angry_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(0)

for kuva in os.listdir(happy_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(happy_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(1)

for kuva in os.listdir(sad_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(sad_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(2)

data = torch.stack(data)
labels = torch.tensor(labels)
labels = F.one_hot(labels)

In [17]:
print(data.shape)
print(labels.shape)

torch.Size([288, 3, 32, 32])
torch.Size([288, 3])


In [18]:
from torch.utils.data import DataLoader, TensorDataset, random_split

labeled_data = TensorDataset(data, labels)
train_len = int(0.8*len(labeled_data))
test_len = len(labeled_data) - train_len

train_data, test_data = random_split(labeled_data, [train_len, test_len])


loaders = {
    "train": DataLoader(train_data, 
                        batch_size= 16, 
                        shuffle=True,),
    "test": DataLoader(test_data,
                       batch_size=16,
                       shuffle=True)
}

In [19]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x168efe900>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x168efc6b0>}

In [20]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [58]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(512, 100)
        self.fc2 = nn.Linear(100, 3)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)),2))
        x = x.view(-1, 512)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return F.softmax(x)

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        target = target.to(float)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            target = target.to(float)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim = True)
            target = target.argmax(dim=1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')


In [89]:
for epoch in range(1,41):
    train(epoch)
test()

/var/folders/s6/9r2gng9n28jfv8k1p2d_82rr0000gn/T/ipykernel_37342/1510074135.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/230 (0%)]	0.613945
Train Epoch: 1 [80/230 (33%)]	0.676591
Train Epoch: 1 [160/230 (67%)]	0.671464
Train Epoch: 2 [0/230 (0%)]	0.677881
Train Epoch: 2 [80/230 (33%)]	0.560108
Train Epoch: 2 [160/230 (67%)]	0.670782
Train Epoch: 3 [0/230 (0%)]	0.740061
Train Epoch: 3 [80/230 (33%)]	0.649685
Train Epoch: 3 [160/230 (67%)]	0.748585
Train Epoch: 4 [0/230 (0%)]	0.551737
Train Epoch: 4 [80/230 (33%)]	0.613993
Train Epoch: 4 [160/230 (67%)]	0.890822
Train Epoch: 5 [0/230 (0%)]	0.675857
Train Epoch: 5 [80/230 (33%)]	0.552961
Train Epoch: 5 [160/230 (67%)]	0.676445
Train Epoch: 6 [0/230 (0%)]	0.682208
Train Epoch: 6 [80/230 (33%)]	0.625222
Train Epoch: 6 [160/230 (67%)]	0.755109
Train Epoch: 7 [0/230 (0%)]	0.551445
Train Epoch: 7 [80/230 (33%)]	0.672103
Train Epoch: 7 [160/230 (67%)]	0.613945
Train Epoch: 8 [0/230 (0%)]	0.612203
Train Epoch: 8 [80/230 (33%)]	0.676445
Train Epoch: 8 [160/230 (67%)]	0.609719
Train Epoch: 9 [0/230 (0%)]	0.738775
Train Epoch: 9 [80/230 (33%)]	0.613